# I. Import libraries <a name="I"></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os 
import warnings

from datetime import datetime, timedelta

from sklearn.model_selection import train_test_split, KFold, cross_validate, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error

import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.exponential_smoothing.ets import ETSModel

In [2]:
# Turn off warnings
warnings.filterwarnings("ignore")

# Set printed decimal limit
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plot theme
plt.style.use('seaborn')
plt.rcParams['figure.figsize'] = (16, 4)
custom_colors = ['#512d6d', '#e6a2b4', '#6b7d96', '#b3cde0']
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=custom_colors)

# II. Import data <a name="II"></a>

## 1. Download data  <a name="II.1"></a>

In [3]:
file_id = "1atOZX3YXmxx-_QebbTfndeC6U_DPTL2e" # ID of the file on Google Drive
file_name = 'Updated_data_2021&2022.csv'

%run download.ipynb

## 2. Import data <a name="II.2"></a>

In [4]:
# File path
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
file_path = os.path.join(parent_dir, 'Data',file_name)

df = pd.read_csv(file_path)
df.head()

,DATE_DIM,DAY_OF_WEEK,BET_ACCOUNT_NUM_HASH,AGE,AGE_BAND,GENDER,TENURE_IN_DAYS,RESIDENTIAL_STATE,FOB_RACING_TURNOVER,FOB_SPORT_TURNOVER,PARI_RACING_TURNOVER,PARI_SPORT_TURNOVER,TOTAL_TURNOVER,DIVIDENDS_PAID,GROSS_MARGIN,TICKETS
0,2021-01-01,Fri,13154,67.00,65+,M,11846,WA,37.00,NaN,1081.00,NaN,1118.00,443.55,271.25,288
1,2021-01-01,Fri,18379,54.00,45-54,M,1884,WA,40.00,NaN,NaN,NaN,40.00,0.00,40.00,1
2,2021-01-01,Fri,559232,63.00,55-64,M,2866,WA,NaN,NaN,12.00,NaN,12.00,9.50,2.04,5
3,2021-01-01,Fri,698904,69.00,65+,M,2100,WA,NaN,NaN,1223.50,NaN,1223.50,267.91,245.12,40
4,2021-01-01,Fri,762921,67.00,65+,M,4766,WA,NaN,NaN,17.50,NaN,17.50,0.00,3.50,5


# III. Clean data <a name="III"></a>

Based on our exploratory data analysis, we apply the similar steps to clean the data.

In [ ]:
def clean_data(df):
    # DATE_DIM: datetime
    df['DATE_DIM'] = pd.to_datetime(df['DATE_DIM'], format='%Y-%m-%d')

    # BET_ACCOUNT_NUM_HASH: string
    df['BET_ACCOUNT_NUM_HASH'] = df['BET_ACCOUNT_NUM_HASH'].astype('O')

    # Impute AGE column
    df['AGE'].fillna(44, inplace=True)

    # More than zero
    df = df[df['TOTAL_TURNOVER'] > 0]
    
    # Drop redundant columns
    df.drop(['DAY_OF_WEEK', 'AGE'], axis=1, inplace=True)

    # Create RACING_TURNOVER column
    df['RACING_TURNOVER'] = df[['FOB_RACING_TURNOVER', 'PARI_RACING_TURNOVER']].sum(axis=1)
    
    return df.set_index('DATE_DIM')

df = clean_data(df)
df.head()

# IV. Aggregate and feature engineering <a name="IV"></a>

We create a dataframe containing all demographic categorical factors: `AGE_BAND`, `GENDER`, `RESIDENTIAL_STATE`

In [ ]:
# Create customer demographic info dataframe
def cus_aggregate(df):
    cus_df = df.groupby('BET_ACCOUNT_NUM_HASH').agg({'AGE_BAND':min, 'GENDER': min, 'RESIDENTIAL_STATE':min})
    return cus_df

cus_df = cus_aggregate(df)
cus_df.head()

We extract behaviorial variables that could impact future spending:
- Frequency ([0-1])
- Racing spending ratio ([0-1])
- Dividends paid ratio (>0)
- Average turnover per day
- Average tickets purchased per day

We go back 1 week, 4 weeks and 12 weeks. Along with demographic data, here is the list of columns to be created and treated as independent variables.

Out predicted (dependent) variables will be the average turnover per day for the next 4 week.

| Column | Description |
|-----------------|-----------------|
| AGE_BAND | Customer’s age band as of Wager date | 
| GENDER | Customer’s gender (M, F, U) | 
| RESIDENTIAL_STATE | Residential state where the customer resides | 
| AVG_FREQ_12 | Betting frequency of the last 12 weeks [0-1] |
| RACING_RATIO_12 | Racing spending ratio of the last 12 weeks [0-1]|
| AVG_TURNOVER_12 | Average turnover per day for the last 12 weeks|
| DIVIDENDS_RATIO_12 |Dividends paid of the last 12 weeks|
| AVG_TICKETS_12 |Average tickets purchased per day for the last 12 weeks|
| AVG_FREQ_4 | Betting frequency of the last 4 weeks [0-1] |
| RACING_RATIO_4 | Racing spending ratio of the last 4 weeks [0-1]|
| AVG_TURNOVER_4 | Average turnover per day for the last 4 weeks|
| DIVIDENDS_RATIO_4 |Dividends paid of the last 4 weeks|
| AVG_TICKETS_4 |Average tickets purchased per day for the last 4 weeks|
| AVG_FREQ_1 | Betting frequency of the last week [0-1] |
| RACING_RATIO_1 | Racing spending ratio of the last week [0-1]|
| AVG_TURNOVER_1 | Average turnover per day for the last week|
| DIVIDENDS_RATIO_1 |Dividends paid of the last week|
| AVG_TICKETS_1 |Average tickets purchased per day for the last week|
| AVG_TURNOVER |Average turnover per day for the next 4 weeks|

In [ ]:
def weekly_aggregate(df, date='2021-05-10', weeks=4):
    # Filtered period
    past_date = (datetime.strptime(date, '%Y-%m-%d') - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    
    # Aggregate
    agg_df = df[(df.index >= past_date ) & (df.index < date)].groupby('BET_ACCOUNT_NUM_HASH').agg({
        'BET_ACCOUNT_NUM_HASH': np.size,
        'TENURE_IN_DAYS' : max,
        'RACING_TURNOVER': sum,
        'TOTAL_TURNOVER' : sum,
        'DIVIDENDS_PAID' : sum,
        'TICKETS' : sum
    })
    
    # Create ratio columns
    agg_df['RACING_TURNOVER'] = agg_df['RACING_TURNOVER'] / agg_df['TOTAL_TURNOVER']
    agg_df['DIVIDENDS_PAID'] = agg_df['DIVIDENDS_PAID'] / agg_df['TOTAL_TURNOVER']

    # Create average columns
    agg_df[['BET_ACCOUNT_NUM_HASH', 'TOTAL_TURNOVER', 'TICKETS']] = agg_df[['BET_ACCOUNT_NUM_HASH', 'TOTAL_TURNOVER', 'TICKETS']] / (weeks*7)
    
    agg_df.columns = ['AVG_FREQ_' + str(weeks), 
                     'TENURE_IN_DAYS_' + str(weeks), 
                     'RACING_RATIO_' + str(weeks),
                     'AVG_TURNOVER_' + str(weeks),
                     'DIVIDENDS_RATIO_' + str(weeks),
                     'AVG_TICKETS_' + str(weeks)]
    
    return agg_df

def total_aggregate(df, date='2021-05-10'):
    # Prediction (4 weeks after the current week)
    future_date = (datetime.strptime(date, '%Y-%m-%d') + timedelta(weeks=4)).strftime('%Y-%m-%d')
    pred = df[ (df.index >= date ) & (df.index < future_date)].groupby('BET_ACCOUNT_NUM_HASH').TOTAL_TURNOVER.sum().to_frame() / 28
    pred.columns = ['AVG_TURNOVER']
    
    # Aggregate
    train_12 = weekly_aggregate(df,date=date, weeks=12)
    train_4 = weekly_aggregate(df,date=date, weeks=4)
    train_1 = weekly_aggregate(df,date=date, weeks=1)
    
    # Filter new customers
    train_12 = train_12[train_12['TENURE_IN_DAYS_12'] >= 84]
    
    # Join data
    train = pd.merge(train_12, train_4, left_index=True, right_index=True, how='left')
    train = pd.merge(train, train_1, left_index=True, right_index=True, how='left')
    train = pd.merge(train, pred, left_index=True, right_index=True, how='left')

    # Drop TENURE_IN_DAYS columns
    train.drop(['TENURE_IN_DAYS_12', 'TENURE_IN_DAYS_4', 'TENURE_IN_DAYS_1'], axis=1, inplace=True)
    
    # Fill na
    train.fillna(0, inplace=True)
    
    return train

In [ ]:
# Choose 2021-05-10'
date='2021-05-10'
train = total_aggregate(df, date=date)

# Join with cus_df to get categorical data
train = pd.merge(train, cus_df, left_index=True, right_index=True, how='left')

# Preview
train.head()

# V. Predictive models

In [ ]:
# Correlation matrix
corr_df = train.corr()

# Visualize
plt.figure(figsize=(12,8))

cmap = sns.light_palette("#512d6d", as_cmap=True)
sns.heatmap(corr_df, cmap=cmap, vmin=0, vmax=1 , annot=True, fmt=".2f")
plt.title("Correllation Heatmap")

plt.plot();

## 1. Regression models

In [ ]:
# Create X and y
X = train.drop('AVG_TURNOVER', axis=1)
y = train['AVG_TURNOVER']

# Evaluation dataframe
eva_df = pd.DataFrame(columns = ["Model", "MSE", "R2"])

In [ ]:
# Choose independent variables
cat_cols = ['AGE_BAND', 'GENDER', 'RESIDENTIAL_STATE']

num_cols = ['AVG_FREQ_12','RACING_RATIO_12','AVG_TURNOVER_12','DIVIDENDS_RATIO_12', 'AVG_TICKETS_12', 'AVG_FREQ_4', 'RACING_RATIO_4', 
            'AVG_TURNOVER_4','DIVIDENDS_RATIO_4','AVG_TICKETS_4','AVG_FREQ_1','RACING_RATIO_1','AVG_TURNOVER_1','DIVIDENDS_RATIO_1','AVG_TICKETS_1']

# Transform columns
one_hot_encoder = OneHotEncoder(sparse = False)

full_pipeline = ColumnTransformer([
    ("cat", one_hot_encoder, cat_cols),
    ("num", "passthrough", num_cols),
])

In [ ]:
# Transform
X_trans = full_pipeline.fit_transform(X)
y = y.values.reshape(-1,1)

### a. Simple linear regression with statsmodel 

In [ ]:
# Add constant term
X_trans = sm.add_constant(X_trans)

# train and val set
X_train, X_val, y_train, y_val = train_test_split(X_trans, y, test_size=0.2, random_state=42)

# Fit model
model = sm.OLS(y_train, X_train) 
results = model.fit()

results.summary()

In [ ]:
# Evaluation metrics
mse = results.mse_resid
R2 = results.rsquared

eva_df = eva_df.append({"Model": "Simple Linear Regression", "MSE" : mse, "R2" : R2},ignore_index=True)
eva_df

Positive points of the model:
- Good fit: R-squared and Adj. R-squared are relatively high
- No autoregression ( Durbin-Watson close to 2)

Problems with the model:
- Multicolinearity (high Cond. No.)
- Insignificant variables
- Heteroskedascity
- Residuals significently deviate from normality

In [ ]:
# Predict
y_pred = results.predict(X_val)
e = y_val.flatten() - y_pred

# Visualize
fig, axes = plt.subplots(1,2, figsize=(16,6))

axes[0].scatter(y_val, y_pred)
axes[0].set_xlabel("Real value")
axes[0].set_ylabel("Predicted value")
axes[0].set_title("Scatter plot: Real turnover vs. Predicted turnover") 

axes[1].scatter(y_val, e)
axes[1].set_xlabel("Real value")
axes[1].set_ylabel("Residuals")
axes[1].set_title("Scatter plot: Real turnover vs. Residuals") 

### b. With sklearn and cross validation

We use k-fold cross validation to eliminate selection bias. MSE above might be more positive because the validation set is within lower range.

In [ ]:
# Transform
X_trans = full_pipeline.fit_transform(X)

In [ ]:
# Fit model
lin_reg = LinearRegression(fit_intercept=True)

# K-fold cross validation with k=10
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(lin_reg, X_trans, y, cv=kfold, scoring = ['neg_mean_squared_error','r2'] )

In [ ]:
# Adding metrics to the evaluation dataframe
mse = -scores['test_neg_mean_squared_error'].mean()
R2 = scores['test_r2'].mean()

eva_df = eva_df.append({"Model": "Simple Linear Regression with cross validation", "MSE" : mse, "R2" : R2},ignore_index=True)
eva_df

### c. Regularization with LASSO and Ridge Regression

#### LASSO Regression

LASSO regression, also known as L1 regularization, is a popular technique in statistical modeling and machine learning used for variable selection and regularization. It is an extension of linear regression that adds a penalty term to the ordinary least squares objective function. It helps eliminating insignificant variables.

In [ ]:
# Create a model instance
lasso_reg = Lasso()

# Define the alpha values to be tested
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

# GridSearchCV
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
lasso_grid = GridSearchCV(estimator=lasso_reg, param_grid={'alpha': alphas}, cv=kfold, return_train_score=True)

# Fit
lasso_grid.fit(X_trans,y)

# Alpha
alpha = lasso_grid.best_params_['alpha']
alpha

In [ ]:
# K-fold cross validation with k=10
lasso_reg = Lasso(alpha=alpha)

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(lasso_reg, X_trans, y, cv=kfold, scoring = ['neg_mean_squared_error','r2'] )

In [ ]:
# Adding metrics to the evaluation dataframe
mse = -scores['test_neg_mean_squared_error'].mean()
R2 = scores['test_r2'].mean()

eva_df = eva_df.append({"Model": "LASSO regression", "MSE" : mse, "R2" : R2},ignore_index=True)
eva_df

#### Ridge Regression

Ridge regression, also known as L2 regularization, is another widely used technique for linear regression that addresses the limitations of ordinary least squares. It is similar to lasso regression but uses a different penalty term. It shrinks down insignificant variables.

In [ ]:
# Create a model instance
ridge_reg = Ridge()

# Define the alpha values to be tested
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

# GridSearchCV
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
ridge_grid = GridSearchCV(estimator=ridge_reg, param_grid={'alpha': alphas}, cv=kfold, return_train_score=True)

# Fit
ridge_grid.fit(X_trans,y)

# Alpha
alpha = ridge_grid.best_params_['alpha']
alpha

In [ ]:
# K-fold cross validation with k=10
ridge_reg = Ridge(alpha=alpha)

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(ridge_reg, X_trans, y, cv=kfold, scoring = ['neg_mean_squared_error','r2'] )

In [ ]:
# Adding metrics to the evaluation dataframe
mse = -scores['test_neg_mean_squared_error'].mean()
R2 = scores['test_r2'].mean()

eva_df = eva_df.append({"Model": "Ridge regression", "MSE" : mse, "R2" : R2},ignore_index=True)
eva_df

### d. Linear Regression with PCA transformation

Principal component analysis (PCA) is a popular algorithm to reduce data dimensions. It helps eradicating multicollinearity.

In [ ]:
# Standard scaling before PCA
std_scaler = StandardScaler()
X_scaled = std_scaler.fit_transform(X_trans)

In [ ]:
# PCA
pca = PCA()
pca.fit(X_scaled)

X_pca = pca.transform(X_scaled)

In [ ]:
# Explained variance ratio
explained_var_ratio = pca.explained_variance_ratio_

# Visualize
plt.figure(figsize=(8,6))
plt.plot(range(1,28),np.cumsum(explained_var_ratio))
plt.title('Cumulative explained variance')
plt.xlabel('Component')
plt.ylabel('Explained Variance')

In [ ]:
# We choose number of components to be 14 that explains 90% of the variance of the original data
X_pca = X_pca[:,:14]

In [ ]:
# Fit model
lin_reg = LinearRegression(fit_intercept=True)

# K-fold cross validation with k=10
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(lin_reg, X_pca, y, cv=kfold, scoring = ['neg_mean_squared_error','r2'] )

In [ ]:
# Adding metrics to the evaluation dataframe
mse = -scores['test_neg_mean_squared_error'].mean()
R2 = scores['test_r2'].mean()

eva_df = eva_df.append({"Model": "Linear Regression with PCA", "MSE" : mse, "R2" : R2},ignore_index=True)
eva_df

### e. Regression with Log Transformation

In [ ]:
fig, axes = plt.subplots(3,5,figsize=(16,12))

for i in range(3):
    for j in range(5):
        axes[i,j].scatter(X.iloc[:,i*5 + j], y)
        axes[i,j].set_title(X.columns[i*5+ j]) 
        axes[i,j].set_yticks([])

In [ ]:
X_log = np.log(X[['AVG_TURNOVER_12','AVG_TURNOVER_4','AVG_TURNOVER_1']]+1)
y_log = np.log(y+1)

In [ ]:
model = sm.OLS(y_log, X_log) 
results = model.fit()

results.summary()

In [ ]:
y_pred = np.exp(results.predict(X_log))-1
mse = mean_squared_error(y.flatten(), y_pred)
R2 = r2_score(y.flatten(), y_pred)

eva_df = eva_df.append({"Model": "Log-Log Regression", "MSE" : mse, "R2" : R2},ignore_index=True)
eva_df

## 2. Regression tree

A decision tree regressor works by constructing a tree-like model where each internal node represents a decision based on a specific feature and a corresponding threshold value. The leaf nodes of the tree contain the predicted output values.

In [ ]:
tree_reg = DecisionTreeRegressor()

param_grid = {
    'max_depth' : range(3,26),
    'min_weight_fraction_leaf' : [0.03,0.05,0.10]
}

tree_reg_grid = GridSearchCV(estimator=tree_reg, param_grid=param_grid, cv=10)
tree_reg_grid.fit(X_trans, y)

In [ ]:
best_params = tree_reg_grid.best_params_
best_params

In [ ]:
# K-fold cross validation with k=10
tree_reg = DecisionTreeRegressor(**best_params)

kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(tree_reg, X_trans, y, cv=kfold, scoring = ['neg_mean_squared_error','r2'] )

In [ ]:
# Add to 
mse = -scores['test_neg_mean_squared_error'].mean()
R2 = scores['test_r2'].mean()

eva_df = eva_df.append({"Model": "Decision tree regressor", "MSE" : mse, "R2" : R2},ignore_index=True)
eva_df

## 3. Random forest

In [ ]:
rf_reg = RandomForestRegressor(n_estimators=500, max_depth= 10, min_weight_fraction_leaf= 0.05, random_state=42)
rf_reg.fit(X_trans, y)

In [ ]:
y_pred = rf_reg.predict(X_trans)
mse = mean_squared_error(y, y_pred)
R2 = r2_score(y, y_pred)

In [ ]:
eva_df = eva_df.append({"Model": "Random forest regressor", "MSE" : mse, "R2" : R2},ignore_index=True)
eva_df

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(y,y_pred)

# VI. Time series model

## 1. Exponential Smoothing

Triple Exponential Smoothing, also known as Holt-Winters' Triple Exponential Smoothing, is a time series forecasting method that extends the concept of exponential smoothing to capture both trend and seasonality in the data. It is a popular technique used in various industries to make accurate predictions for time-dependent data.

In this particular case, daily trend can be acting at high variance. Therefore, we choose weekly granularity level. We run a triple exponential smoothing model with trend factor on all historical data and predict the next 4 week turnover spending of a customer.

In [ ]:
date = '2021-05-10'
future_date = (datetime.strptime(date, '%Y-%m-%d') + timedelta(weeks=4)).strftime('%Y-%m-%d')

In [ ]:
train_df = df[df.index < date]
test_df = df[(df.index >= date) & (df.index < future_date)]

In [ ]:
cus = train_df.groupby('BET_ACCOUNT_NUM_HASH',as_index=False).TENURE_IN_DAYS.max()
legitimate_cus = cus.loc[cus['TENURE_IN_DAYS'] >= 28, 'BET_ACCOUNT_NUM_HASH']

In [ ]:
train_df = train_df[train_df['BET_ACCOUNT_NUM_HASH'].isin(legitimate_cus)]
test_df = test_df[test_df['BET_ACCOUNT_NUM_HASH'].isin(legitimate_cus)]

In [ ]:
res = pd.DataFrame(columns = ['BET_ACCOUNT_NUM_HASH', 'real', 'pred'])
X_test = test_df.groupby('BET_ACCOUNT_NUM_HASH').TOTAL_TURNOVER.sum()

In [ ]:
for i in legitimate_cus: 
    cus_df = train_df[train_df['BET_ACCOUNT_NUM_HASH'] == i]

    X_train = cus_df.resample('W').TOTAL_TURNOVER.sum()

    X_train = X_train.reindex(pd.date_range(start='2021-01-03',end=future_date, freq="W")).fillna(0)

    exp_smth = ETSModel(X_train, trend = "add", freq='W')

    result = exp_smth.fit()

    start = X_train.index[-1] + pd.DateOffset(7)
    end = X_train.index[-1] + pd.DateOffset(28)

    X_forecast = result.predict(start=start, end=end)

    try:
        real = X_test[i]
    except:
        real = 0

    pred = X_forecast.sum()

    res = res.append({'BET_ACCOUNT_NUM_HASH' : i, 'real':real, 'pred': pred}, ignore_index=True)